In [34]:
import requests
import os

import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

FSbrukar = os.environ["FSbrukar"]
FSpassord = os.environ["FSpassord"]

def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return 


In [2]:
query_termin = """
query MyQuery($arstall: [Int!], $terminbetegnelse: [ID!]) {
  terminer(
    filter: {eierOrganisasjonskode: "0203", arstall: $arstall, terminbetegnelser: $terminbetegnelse}
  ) {
    nodes {
      id
      betegnelse {
        kode
      }
      arstall
    }
  }
}
"""


query_emneid = """
query MyQuery($emnekode: [String!]) {
  emner(filter: {eierInstitusjonsnummer: "0203", emnekoder: $emnekode}) {
    nodes {
      id
    }
  }
}
"""

query_vurderingsenheter = """
query MyQuery ($emne_id: [ID!], $termin_id: [ID!], $emnekode: String!) {
  vurderingsenheter(
    filter: {eierOrganisasjonskode: "203", emner: $emne_id, terminer: $termin_id}
  ) {
    nodes {
      vurderingsmeldinger {
        student {
          vurderingsforsokForEmner(emnekoder: {emnekode: $emnekode, versjonskode: "1"}) {
            antallTellendeForsok
          }
          studentnummer
        }
      }
    }
  }
}
"""

query_undervisningsaktiviteter = """
query MyQuery($emne_id: [ID!], $termin_id: [ID!]) {
  undervisningsenheter(
    filter: {eierOrganisasjonskode: "203", emner: $emne_id, terminer: $termin_id}
  ) {
    nodes {
      undervisningsaktiviteter {
        nodes {
          id
        }
      }
    }
  }
}
"""

query_partiplassering = """
query MyQuery($undervisningsaktiviteter: [ID!]) {
undervisningspartiplasseringer(
    filter: {eierOrganisasjonskode: "203", emneSkalEksporteresTilLms: true, undervisningsaktiviteter: $undervisningsaktiviteter}
    first: 1000
  ) {
    nodes {
      student {
        studentnummer
      }
      undervisningsaktivitet {
        kode
      }
    }
  }
}
"""

In [13]:
emnekode = "BYG102"
år = 2025
termin = "VÅR"


In [14]:

if termin == "VÅR":
    terminbetegnelse = "MTE3OjIwMyxWw4VS"
else:
    terminbetegnelse = "MTE3OjIwMyxIw5hTVA"
variable = {'arstall': år, 'terminbetegnelse': terminbetegnelse}
svar = query_FS_graphql(query_termin, variable)
termin_id = svar['data']['terminer']['nodes'][0]['id']
print(termin_id)

MTE2OjIwMywyMDI1LFbDhVI


In [15]:

variable = {'emnekode': emnekode, }
svar = query_FS_graphql(query_emneid, variable)
emne_id = svar['data']['emner']['nodes'][0]['id']
print(emne_id)

MjA6MjAzLDIwMyxCWUcxMDIsMQ


In [16]:
variable = {'emne_id': emne_id, 'termin_id': termin_id}
svar = query_FS_graphql(query_undervisningsaktiviteter, variable)
temp = svar['data']['undervisningsenheter']['nodes'][0]['undervisningsaktiviteter']['nodes']
UA_liste = []
for ua in temp:
    UA_liste.append(ua['id'])
print(UA_liste)

['MTIwOjIwMywyMDMsQllHMTAyLDEsMjAyNSxWw4VSLDEsMA', 'MTIwOjIwMywyMDMsQllHMTAyLDEsMjAyNSxWw4VSLDEsMQ', 'MTIwOjIwMywyMDMsQllHMTAyLDEsMjAyNSxWw4VSLDEsMw', 'MTIwOjIwMywyMDMsQllHMTAyLDEsMjAyNSxWw4VSLDEsNA']


In [17]:
variable = {'undervisningsaktiviteter': UA_liste}
svar = query_FS_graphql(query_partiplassering, variable)
partiplasseringer = svar['data']['undervisningspartiplasseringer']['nodes']
plassert = []
for p in partiplasseringer:
    plassert.append([p['student']['studentnummer'], p['undervisningsaktivitet']['kode']])
print(len(plassert))


157


In [18]:
variable = {'emne_id': emne_id, 'termin': termin_id, 'emnekode': emnekode}
svar = query_FS_graphql(query_vurderingsenheter, variable)
vurderingsmeldingar = svar['data']['vurderingsenheter']['nodes']
ekstra_vurderingsmeldt = []
for v in vurderingsmeldingar:
    if len(v['vurderingsmeldinger']) > 0:
        for s in v['vurderingsmeldinger']:
            if len(s['student']['vurderingsforsokForEmner']) > 0:
                # print(f"{s['student']['studentnummer']} - {s['student']['vurderingsforsokForEmner'][0]['antallTellendeForsok']}")
                ekstra_vurderingsmeldt.append(f"{s['student']['studentnummer']}@stud.hvl.no")
print(len(ekstra_vurderingsmeldt))

21


In [19]:
ekstra_vurderingsmeldt

['594287@stud.hvl.no',
 '584909@stud.hvl.no',
 '601860@stud.hvl.no',
 '594287@stud.hvl.no',
 '584909@stud.hvl.no',
 '601860@stud.hvl.no',
 '798386@stud.hvl.no',
 '594287@stud.hvl.no',
 '584909@stud.hvl.no',
 '601860@stud.hvl.no',
 '798386@stud.hvl.no',
 '594287@stud.hvl.no',
 '584909@stud.hvl.no',
 '601860@stud.hvl.no',
 '798386@stud.hvl.no',
 '673182@stud.hvl.no',
 '594287@stud.hvl.no',
 '584909@stud.hvl.no',
 '601860@stud.hvl.no',
 '798386@stud.hvl.no',
 '673182@stud.hvl.no']

For å sjekke om "query_partiplassering" spør etter dei rette data kan eg hente ned data som vet sendt frå FS til Canvas kvar natt, og så plukke ut alle som er meldt på i emnet:

In [ ]:
enrollments = pd.read_csv("enrollments.csv")
enrollments = enrollments.copy()
enrollments.dropna(subset=['section_id'], inplace=True)
emnestreng = f"{emnekode}_1_{år}_{termin}_1_"
undervisingsmeldt = enrollments[enrollments['section_id'].str.contains(emnestreng)]

161


C:\Users\AKV\AppData\Local\Temp\ipykernel_21748\2459036833.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  enrollments = pd.read_csv("enrollments.csv")


In [32]:
partiplassert = pd.DataFrame(plassert, columns=['studentnummer', 'seksjon'])
partiplassert['seksjon'].value_counts()

seksjon
3    81
1    76
Name: count, dtype: int64

In [33]:
undervisingsmeldt = undervisingsmeldt.copy()
undervisingsmeldt['seksjon'] = undervisingsmeldt['section_id'].apply(lambda x: x[-1:])
undervisingsmeldt['seksjon'].value_counts()

seksjon
3    83
1    78
Name: count, dtype: int64

Som eit alternativ til å hente frå SIS-import kvar natt (som kan vere ei stor fil) kan eg hente påmeldte studentar drekte frå emnet. Men då må eg vite den interne koden for emnet i Canvas. Den finn eg ikkje frå `emnestreng`.

In [44]:
url = f"https://hvl.instructure.com/api/v1/accounts/1?search_term={emnestreng}"
respons = requests.get(url, headers=headers, params=params)
emne = respons.json()

In [45]:
for e in emne:
    print(e['id'], e['name'])

TypeError: string indices must be integers, not 'str'

In [46]:
emne

{'id': 1,
 'name': 'Høgskulen på Vestlandet',
 'workflow_state': 'active',
 'parent_account_id': None,
 'root_account_id': None,
 'uuid': 'KjYXFlWyqIP306jMLrOxOE6QSU1S8oF5m5tfbXyd',
 'default_storage_quota_mb': 524,
 'default_user_storage_quota_mb': 524,
 'default_group_storage_quota_mb': 524,
 'default_time_zone': 'Europe/Copenhagen',
 'course_template_id': None}